In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

# Sample corpus
data = "Deep learning is amazing. Deep learning builds intelligent systems."

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

sequences = []
for i in range(1, len(encoded)):
    seq = encoded[:i+1]
    sequences.append(seq)

max_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=len(tokenizer.word_index)+1)

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10, input_length=max_len-1),
    SimpleRNN(50),
    Dense(len(tokenizer.word_index)+1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=0)

def predict_next_word(model, tokenizer, text, max_len):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
    y_pred = model.predict(encoded, verbose=0)
    predicted_word = tokenizer.index_word[np.argmax(y_pred)]
    return predicted_word

print("\n--- Step by Step Predictions ---")
words = data.split()
for i in range(1, len(words)):
    input_seq = ' '.join(words[:i])
    next_word = predict_next_word(model, tokenizer, input_seq, max_len)
    print(f"[{input_seq}] -> '{next_word}'")



--- Step by Step Predictions ---
[Deep] -> 'learning'
[Deep learning] -> 'is'
[Deep learning is] -> 'amazing'
[Deep learning is amazing.] -> 'deep'
[Deep learning is amazing. Deep] -> 'learning'
[Deep learning is amazing. Deep learning] -> 'builds'
[Deep learning is amazing. Deep learning builds] -> 'intelligent'
[Deep learning is amazing. Deep learning builds intelligent] -> 'systems'


In [12]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# corpus must be your cleaned Shakespeare text (list of lines)
# Example:
corpus = [
     "shall i compare thee to a summers day",
     "thou art more lovely and more temperate",
     "rough winds do shake the darling buds of may"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Generate n-gram sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

# Split into predictors and label
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Model
model = Sequential([
    Embedding(total_words, 100, input_length=max_len-1),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 🔹 This line prints the epoch logs like in your screenshot
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0476 - loss: 3.1769
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0952 - loss: 3.1674
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0952 - loss: 3.1576
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0952 - loss: 3.1470
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.0952 - loss: 3.1355
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.0952 - loss: 3.1224
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.0952 - loss: 3.1072
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0952 - loss: 3.0893
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.0952 - loss: 3.0678
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.0952 - loss: 3.0418
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.0952 - loss: 3.0103
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0952 - loss: 2.9733